In [2]:
import requests
from requests.auth import HTTPBasicAuth
import pandas as pd
import pytz
import ipywidgets as widgets
import datetime
import numpy as np

In [339]:
def gen_url(
    telemetry='min', BB3='min', CI='min', CT='min', O2='min',
    start=datetime.datetime.now() - datetime.timedelta(days=30),
    end=datetime.datetime.now()
    ):

    # Returns string for URL based on group variable selection and date

    # Base strings and querry dictionaries
    group_options = {
    'telemetry': telemetry, 
    'BB3': BB3, 
    'CI': CI, 
    'CT': CT, 
    'O2': O2
    }

    querry = {
    'telemetry': {
    'all': ['GPSTimeStamp', '2CLon', '2CLat', '2CSpeed', '2CTrack', '2CHeading', '2CPitch', '2CRoll', '2CWindAppSpeed', '2CWindDegOffBow','2CWindTrueSpeed', '2CWindTrueDir', '2CPressureInches', '2CAirTemp'],
    'min': ['GPSTimeStamp', '2CLon', '2CLat',],
    'atmo': ['GPSTimeStamp', '2CLon', '2CLat', '2CPressureInches', '2CAirTemp', '2CWindAppSpeed', '2CWindDegOffBow', '2CWindTrueSpeed', '2CWindTrueDir',]
    },
    'BB3': {
    'all': ['2CBB3+%5BTime+UTC%5D', '2CBb%28470%29+%5Bcounts%5D', '2CBb%28532%29+%5Bcounts%5D', '2CBb%28650%29+%5Bcounts%5D', '2CBb%28470%29+%5BNTU%5D', '2CBb%28532%29+%5BNTU%5D', '2CBb%28650%29+%5BNTU%5D'],
    'min': ['2CBb%28470%29+%5BNTU%5D', '2CBb%28532%29+%5BNTU%5D','2CBb%28650%29+%5BNTU%5D']
    },
    'CI': {
    'all': ['2CCI+%5BTime+UTC%5D', '2CChl.+a+%5Bcounts%5D', '2CCDOM+%5Bcounts%5D', '2CPhycocyanin+%5Bcounts%5D', '2CCDOM+%5BQSU%5D', '2CChl.+a+%5Bppb%5D', '2CPhycocyanin+%5Bppb%5D'],
    'min': ['2CCDOM+%5BQSU%5D', '2CChl.+a+%5Bppb%5D', '2CPhycocyanin+%5Bppb%5D']
    },
    'CT': {
    'all': ['2CCT+%5BTime+UTC%5D', '2CConductivity+%5BmS+cm-1%5D', '2CTemperature+%5Bdeg+C%5D'],
    'min': ['2CConductivity+%5BmS+cm-1%5D', '2CTemperature+%5Bdeg+C%5D']
    },
    'O2' : {
    'all': ['2CO2+%5BTime+UTC%5D', '2CO2+Concentration+%5Bmicromolar%5D', '2CO2+Saturation+%5B%25%5D', '2CO2+Temperature+%5Bdeg+C%5D'],
    'min': ['2CO2+Concentration+%5Bmicromolar%5D', '2CO2+Saturation+%5B%25%5D']
    }
    }

    time_dic = {'GPSTimeStamp': 'GPSTimeStamp', '2CBB3+%5BTime+UTC%5D': 'BB3 [Time UTC]', '2CCI+%5BTime+UTC%5D': 'CI [Time UTC]', '2CO2+%5BTime+UTC%5D': 'CT [Time UTC]'}

    root_url = 'http://portal.navocean.com/services/nav.php?req=data&id=VELA'
    fmtt = 'format=csv&output=file'

    token= 'token=5e5c4d86-3fd9-11eb-904e-06ad0ec96835'


    # build string for variable request
    columns = []
    for g in querry.keys():
        if group_options[g]:
            columns.extend(querry[g][group_options[g]])
    columns = 'columns='+'%'.join(columns)

    # build string for date request
    start = dtodt(start)
    start = start.astimezone(pytz.timezone('utc')).strftime('%Y-%m-%d+%H')
    end = dtodt(end)
    end = end.astimezone(pytz.timezone('utc')).strftime('%Y-%m-%d+%H')
    time = 'start='+start+'%3A00%3A00&end='+end+'%3A00%3A00'
    url = '&'.join([root_url, columns, fmtt, time, token]) #, date_vars

    # get date type variables for parser
    date_cols = []
    for time_key in time_dic:
        if time_key in url:
            date_cols.append(time_dic[time_key])
    return url, date_cols

def load_csv(path, date_vars=False):
    parse_dates = date_vars
    date_parser=lambda x: pd.to_datetime(x, errors="coerce")

    return pd.read_csv(path, skipinitialspace=True, parse_dates=parse_dates, date_parser=date_parser,
        index_col='Id').replace(r'^\s*$', np.nan, regex=True)

def get_data(path, date_vars=['GPSTimeStamp']):
    # Output: dataframe:
    # Input: either navocean url, or a lists of filenames
    # Names of date type variables to parse
    if type(path) is str:
        if 'http' in path:
            request = requests.get(path, auth=HTTPBasicAuth('okeechobee', 'cleanwater'), stream=True)
            file = request.raw
        else:
            file = path
        df = load_csv(file, date_vars=date_vars)

    elif type(path) is list:
        li = []
        for f in path:
            df = load_csv(f, date_vars=date_vars)
            li.append(df)
            df = pd.concat(li, axis=0, ignore_index=True)
    df = df[~df['GPSTimeStamp'].isnull()]           
    
    df['local_time'] = df['GPSTimeStamp'].dt.tz_localize('utc').dt.tz_convert(pytz.timezone('US/Eastern'))
#     df.set_index('GPSTimeStamp', inplace=True)
    return df

def dtodt(date):
    if type(date) == datetime.date:
        date = datetime.datetime(year=date.year, month=date.month, day=date.day)
    return date


# Select time frame for sailboat data 

In [344]:
start = widgets.DatePicker(
    description='Start date',
    disabled=False,
    value= datetime.datetime.now() - datetime.timedelta(days=30)
)

end = widgets.DatePicker(
    description='End date',
    disabled=False,
    value=datetime.datetime.now()
)


ui = widgets.HBox([start, end])
display(ui)

def f(start, end):
    df = get_data(gen_url(start=start, end=end)[0])
    return df

dfc = f(start.value, end.value)
# out = widgets.interactive_output(f, {'start': start, 'end': end})

In [345]:
import plotly.express as px  # (version 4.7.0 or higher)
import plotly.graph_objects as go

## Dataset

In [346]:
#QC
df = dfc.copy()
df = df[(df['Lon']>-81.15)&(df['Lon']<-80.51)]
df = df[(df['Lat']<27.24)&(df['Lat']>26.65)]
df = df[df['Lon'].notna()]
df = df[df['Lat'].notna()]
df = df[df['GPSTimeStamp'].notnull()]
df = df[~df['local_time'].isnull()]

# df[df[df.columns[2:-1]]<0] = np.nan
df.loc[df['Conductivity [mS cm-1]']<0.2, 'Conductivity [mS cm-1]']=np.nan
df.loc[df['Conductivity [mS cm-1]']>.5, 'Conductivity [mS cm-1]']=np.nan
df.loc[df['Temperature [deg C]']>40., 'Temperature [deg C]']=np.nan
df.loc[df['Temperature [deg C]']<5., 'Temperature [deg C]']=np.nan

df.head(5)

,GPSTimeStamp,Lon,Lat,Conductivity [mS cm-1],Temperature [deg C],O2 Concentration [micromolar],O2 Saturation [%],Bb(470) [NTU],Bb(532) [NTU],Bb(650) [NTU],CDOM [QSU],Chl. a [ppb],Phycocyanin [ppb],local_time
Id,,,,,,,,,,,,,,
427499,2022-01-01 19:47:34,-80.796425,27.222744,NaN,24.635,0.0,0.0,1.695,1.942,1.698,437.12,224.62,13.22,2022-01-01 14:47:34-05:00
427498,2022-01-01 19:47:03,-80.796432,27.222748,NaN,24.649,0.0,0.0,1.695,1.942,1.698,434.54,217.16,13.09,2022-01-01 14:47:03-05:00
427497,2022-01-01 19:45:18,-80.796425,27.222752,NaN,24.625,0.0,0.0,1.938,1.942,1.698,441.03,239.16,14.68,2022-01-01 14:45:18-05:00
427496,2022-01-01 19:43:29,-80.796417,27.222755,NaN,25.006,0.0,0.0,1.938,2.185,1.941,437.71,246.11,15.79,2022-01-01 14:43:29-05:00
427495,2022-01-01 19:42:58,-80.796440,27.222755,NaN,25.072,0.0,0.0,1.695,1.942,1.698,433.28,185.90,16.10,2022-01-01 14:42:58-05:00


In [156]:
def tsplot(y):
    fig = px.line(df, x='local_time', y=y)
    fig.show()

def scplot(x,y,c):
    fig = px.scatter(df, x=x, y=y, color=c)
    fig.show()
    


# Time series

In [214]:
interactive_plot = widgets.interactive(tsplot, y=df.columns[3:-1])
interactive_plot

interactive(children=(Dropdown(description='y', options=('Conductivity [mS cm-1]', 'Temperature [deg C]', 'O2 …

# Scatter plot

In [342]:
x = widgets.Dropdown(options=df.columns[3:-1], value='Chl. a [ppb]', description='x axis')
y = widgets.Dropdown(options=df.columns[3:-1], value='Phycocyanin [ppb]', description='y axis')
c = widgets.Dropdown(options=df.columns[3:-1], value='Bb(470) [NTU]', description='color')

out = widgets.interactive(scplot, x=x, y=y, c=c)
display(out)

interactive(children=(Dropdown(description='x axis', index=8, options=('Conductivity [mS cm-1]', 'Temperature …

In [195]:
dfc.columns

Index(['GPSTimeStamp', 'Lon', 'Lat', 'Conductivity [mS cm-1]',
       'Temperature [deg C]', 'O2 Concentration [micromolar]',
       'O2 Saturation [%]', 'Bb(470) [NTU]', 'Bb(532) [NTU]', 'Bb(650) [NTU]',
       'CDOM [QSU]', 'Chl. a [ppb]', 'Phycocyanin [ppb]', 'local_time'],
      dtype='object')

# Path plot

In [352]:
df['local_time'].values[0]

numpy.datetime64('2022-01-01T19:47:34.000000000')

In [368]:
varchoice = widgets.Dropdown(options=df.columns[3:-1], value='Chl. a [ppb]', description='Variable')
varchoice

dates = pd.date_range(df['local_time'].values[-1], df['local_time'].values[0], freq='D')
options = [(date.strftime(' %d/%b/%Y '), date) for date in dates]
index = (0, len(options)-1)

selection_range_slider = widgets.SelectionRangeSlider(
    options=options,
    index=index,
    description='Date range',
    orientation='horizontal',
    layout={'width': '600px'}
)
selection_range_slider

# interactive_plot = widgets.interactive(pathplot, )

out = widgets.interactive_output(pathplot, {'var': varchoice, 'date_range': selection_range_slider})
out
# interactive_plot

Output()

In [367]:
def pathplot(var, date_range):
    range_dic = {
    'Conductivity [mS cm-1]':{'min':0.2, 'max':.5},
    'Temperature [deg C]':{'min':5., 'max':40.},
    'O2 Concentration [micromolar]':{'min':0, 'max':200},
    'O2 Saturation [%]':{'min':0, 'max':300},
    'Bb(470) [NTU]':{'min':0, 'max':1000}, 
    'Bb(532) [NTU]':{'min':0, 'max':1000},
    'Bb(650) [NTU]':{'min':0, 'max':1000},
    'CDOM [QSU]':{'min':0, 'max':100},
    'Chl. a [ppb]':{'min':0, 'max':100}, 
    'Phycocyanin [ppb]':{'min':0, 'max':100}
    }
    
    df = df[df['local_time']>date_range.values[0]]
    df = df[df['local_time']<date_range.values[1]]
    fig = px.scatter_mapbox(df, lat="Lat", lon="Lon",
                            color=var, range_color=[range_dic[var]['min'],range_dic[var]['max']],
                            hover_name=var,
    #                         color_discrete_sequence=["fuchsia"], 
                            zoom=10,
                            center={'lat':26.95, 'lon':-80.84},
                            height=850,
                           width=850,
                           )
    fig.update_layout(
        mapbox_style="white-bg",
        mapbox_layers=[
            {
                "below": 'traces',
                "sourcetype": "raster",
                "sourceattribution": "United States Geological Survey",
                "source": [
                    "https://basemap.nationalmap.gov/arcgis/rest/services/USGSImageryOnly/MapServer/tile/{z}/{y}/{x}"
                ]
            },

          ])
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
    fig.show()

In [357]:
end_date

numpy.datetime64('2021-12-21T11:11:50.000000000')